# Machine Learning - Homework
# Machine Learning and Security Research: Function classification





## Import needed libraries

In [1]:
import numpy as np
import pandas as pd
import random
import json
import networkx as nx
from networkx import json_graph

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import *
from sklearn.naive_bayes import *
from sklearn.metrics import confusion_matrix, classification_report
import sklearn.metrics 
from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit, GridSearchCV
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt

print('Libraries imported.')

Libraries imported.


## Load data

Mount Google Drive by following the instructions given at the provided link

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
filename = '/content/drive/MyDrive/nodup_classification/noduplicatedataset.json'
#filename = '/content/drive/MyDrive/nodup_classification/dataset.json'
with open(filename,'r') as f:
    lines = f.readlines()
    dataset = pd.DataFrame([], columns = ['Id', 'Semantic', 'contA', 'contB', 'contC', 'contD', 'contE', 'contF', 'Nodes num', 'Diameter', 'Cycles num', 'Complexity'])
    for item in lines:
      d = json.loads(item)

      nx_graph = json_graph.adjacency_graph(d['cfg'])
      nodes_num = len(nx.nodes(nx_graph))
      edges_num = len(nx.edges(nx_graph))
      parts = nx.components.number_strongly_connected_components(nx_graph)
      diameter = nx.diameter(nx_graph.to_undirected())
      cycles_num = len(nx.cycle_basis(nx_graph.to_undirected()))
      complexity = edges_num - nodes_num +2*parts

      id = d['id']
      semantic = d['semantic']

      contA = 0
      contB = 0
      contC = 0
      contD = 0
      contE = 0
      contF = 0
      groupA = ['mov', 'push', 'pop', 'lea']
      groupB = ['inc', 'dec', 'neg', 'add', 'sub', 'imul', 'idiv', 'sar', 'shl', 'shr']
      groupC = ['xmm']
      groupD = ['or', 'not', 'and']
      groupE = ['call', 'leave', 'ret']
      groupF = ['jmp', 'je', 'jn', 'js', 'jg', 'jl', 'ja', 'jb']

      d['lista_asm'] = d['lista_asm'].replace('[', '')
      d['lista_asm'] = d['lista_asm'].replace(']', '')
      d['lista_asm'] = d['lista_asm'].replace(',', '')
      d['lista_asm'] = d['lista_asm'].replace("'", '')
      for instruction in d['lista_asm'].split(' '):
        for a in groupA:
          if (instruction.find(a) != -1):
            contA += 1
        for b in groupB:
          if (instruction.find(b) != -1):
            contB += 1
        for c in groupC:
          if (instruction.find(c) != -1):
            contC += 1
        for d in groupD:
          if (instruction.find(d) != -1):
            contD += 1
        for e in groupE:
          if (instruction.find(e) != -1):
            contE += 1
        for f in groupF:
          if (instruction.find(f) != -1):
            contF += 1
      dataset2 = pd.DataFrame([[id, semantic, contA, contB, contC, contD, contE, contF, nodes_num, diameter, cycles_num, complexity]], columns = ['Id', 'Semantic', 'contA', 'contB', 'contC', 'contD', 'contE', 'contF', 'Nodes num', 'Diameter', 'Cycles num', 'Complexity'])
      dataset = dataset.append(dataset2, ignore_index= True)
    print(dataset)

         Id    Semantic contA contB  ... Nodes num Diameter Cycles num Complexity
0       828      string    54     5  ...        23        9          8         53
1     11786        math    10     3  ...        15        6          6         35
2     12621  encryption   215   154  ...        18        7         16         33
3     11166        math     7     1  ...         2        1          0          3
4     10432        sort    25     7  ...        22        6         11         45
...     ...         ...   ...   ...  ...       ...      ...        ...        ...
6068  12484        math    17     4  ...         8        4          3         18
6069   7809        sort    52    21  ...        22        6         13         18
6070   9806  encryption   282    95  ...       113       11         87        115
6071    421        math    29     2  ...        11        6          2         23
6072   2008        sort    39    22  ...        10        4          5         18

[6073 rows x 12

## Split data

In [12]:
X_all = dataset
X_all = X_all.drop(['Id', 'Semantic'], axis=1)
y_all = dataset.Semantic

print(X_all.shape)
print(y_all.shape)

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.333, random_state=1)

print("Train: %d - Test: %d" %(X_train.shape[0],X_test.shape[0]))
print(X_train,'\n\n', X_test,'\n\n', y_train,'\n\n', y_test)

(6073, 10)
(6073,)
Train: 4050 - Test: 2023
     contA contB contC contD  ... Nodes num Diameter Cycles num Complexity
4087    46    10     0    16  ...        25        8         13         61
4932    28     7     0     4  ...        12        6          5         16
5904    10     1     2     8  ...         2        1          0          3
1301     2     1    12     4  ...         5        3          1         10
4832    11     1     0     2  ...        15        8          0         28
...    ...   ...   ...   ...  ...       ...      ...        ...        ...
905     50     4     0    16  ...        15        4         12         41
5192    38     9     3    35  ...        19        8          4         37
3980    44     4   109    78  ...        37        8         16         89
235      5     1     3     3  ...         2        1          0          3
5157    47    69     0    52  ...        10        4          7         26

[4050 rows x 10 columns] 

      contA contB contC cont

## Create and fit Model

In [26]:

#model = BernoulliNB().fit(X_train, y_train)
#model = MultinomialNB().fit(X_train, y_train)
model = tree.DecisionTreeClassifier()
#model = svm.SVC(kernel='linear', C=1)
model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

## Evaluation

In [27]:
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[355   5   6   4]
 [  4 818   3   6]
 [ 12   0 161  12]
 [  2   2  10 623]]
              precision    recall  f1-score   support

  encryption       0.95      0.96      0.96       370
        math       0.99      0.98      0.99       831
        sort       0.89      0.87      0.88       185
      string       0.97      0.98      0.97       637

    accuracy                           0.97      2023
   macro avg       0.95      0.95      0.95      2023
weighted avg       0.97      0.97      0.97      2023



Cross validation

In [29]:
cv = ShuffleSplit(n_splits=5, test_size=0.333, random_state=15)
scores = cross_val_score(model, X_all, y_all, cv=cv)
print(scores)
print("Accuracy: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.95254572 0.96440929 0.96490361 0.96490361 0.96836382]
Accuracy: 0.963 (+/- 0.01)


## Load blind test and Prediction

In [28]:
filename = '/content/drive/MyDrive/nodup_classification/blindtest.json'
with open(filename,'r') as f:
    lines = f.readlines()
    newdataset = pd.DataFrame([], columns = ['Id', 'contA', 'contB', 'contC', 'contD', 'contE', 'contF', 'Nodes num', 'Diameter', 'Cycles num', 'Complexity'])
    for item in lines:
      d = json.loads(item)

      nx_graph = json_graph.adjacency_graph(d['cfg'])
      nodes_num = len(nx.nodes(nx_graph))
      edges_num = len(nx.edges(nx_graph))
      parts = nx.components.number_strongly_connected_components(nx_graph)
      diameter = nx.diameter(nx_graph.to_undirected())
      cycles_num = len(nx.cycle_basis(nx_graph.to_undirected()))
      complexity = edges_num - nodes_num +2*parts

      id = d['id']

      contA = 0
      contB = 0
      contC = 0
      contD = 0
      contE = 0
      contF = 0
      groupA = ['mov', 'push', 'pop', 'lea']
      groupB = ['inc', 'dec', 'neg', 'add', 'sub', 'imul', 'idiv', 'sar', 'shl', 'shr']
      groupC = ['xmm']
      groupD = ['or', 'not', 'and']
      groupE = ['call', 'leave', 'ret']
      groupF = ['jmp', 'je', 'jn', 'js', 'jg', 'jl', 'ja', 'jb']

      d['lista_asm'] = d['lista_asm'].replace('[', '')
      d['lista_asm'] = d['lista_asm'].replace(']', '')
      d['lista_asm'] = d['lista_asm'].replace(',', '')
      d['lista_asm'] = d['lista_asm'].replace("'", '')
      for instruction in d['lista_asm'].split(' '):
        for a in groupA:
          if (instruction.find(a) != -1):
            contA += 1
        for b in groupB:
          if (instruction.find(b) != -1):
            contB += 1
        for c in groupC:
          if (instruction.find(c) != -1):
            contC += 1
        for d in groupD:
          if (instruction.find(d) != -1):
            contD += 1
        for e in groupE:
          if (instruction.find(e) != -1):
            contE += 1
        for f in groupF:
          if (instruction.find(f) != -1):
            contF += 1
      newdataset2 = pd.DataFrame([[id, contA, contB, contC, contD, contE, contF, nodes_num, diameter, cycles_num, complexity]], columns = ['Id', 'contA', 'contB', 'contC', 'contD', 'contE', 'contF', 'Nodes num', 'Diameter', 'Cycles num', 'Complexity'])
      newdataset = newdataset.append(newdataset2, ignore_index= True)
    print(newdataset)

        Id contA contB contC  ... Nodes num Diameter Cycles num Complexity
0    10351   264    52    34  ...        93        9         68        101
1    14513    10     2     2  ...         7        5          0         13
2     1767     8     0     0  ...         9        5          0         17
3     9384    65     9     0  ...        38       11         15         90
4     3018    15     3    26  ...         6        4          2         13
..     ...   ...   ...   ...  ...       ...      ...        ...        ...
752   9661     9     5    30  ...        10        5          1         20
753   6933    18    10     0  ...        13        6          8         21
754  11954    31    18     4  ...         5        3          2         11
755   5769   228    32   432  ...       110       19         34        253
756   7957    10     3     9  ...         6        4          1         12

[757 rows x 11 columns]


In [34]:
new_X = newdataset.drop(['Id'], axis=1)
new_y_pred = model.predict(new_X)
for elem in new_y_pred:
  print(elem)

encryption
math
sort
string
math
string
encryption
encryption
sort
math
math
math
sort
sort
sort
math
string
string
math
math
math
string
sort
math
encryption
string
string
math
math
sort
math
sort
sort
sort
string
sort
string
math
sort
string
string
math
sort
math
sort
sort
sort
string
string
string
sort
math
string
string
math
string
math
math
math
math
encryption
sort
math
string
string
sort
math
string
sort
sort
encryption
encryption
string
sort
math
sort
sort
sort
sort
sort
sort
encryption
math
sort
string
math
sort
math
encryption
math
string
string
string
sort
sort
string
math
encryption
encryption
encryption
math
sort
sort
sort
encryption
math
math
math
math
sort
encryption
string
sort
string
string
encryption
math
sort
sort
string
encryption
math
math
encryption
math
encryption
math
string
sort
encryption
encryption
string
sort
math
string
sort
math
string
encryption
math
sort
string
sort
math
sort
encryption
math
sort
encryption
math
math
math
string
string
encryption
string
